In [1]:
"""
Tkinter Tile Match Game with bot (using real images)
------------------------------------------------------
Static 4x4 memory game where you and a bot take turns matching image pairs.
Uses provided real images (apple, orange, camera, basketball, coconut) repeatedly.

Requirements:
 - Python 3.8+
 - Pillow (pip install pillow)

How to run:
    python tk_bot_tile_game.py
"""

import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import random
from functools import partial

# ------------------------- Configuration -------------------------
GRID_ROWS = 4
GRID_COLS = 4
PAIR_COUNT = (GRID_ROWS * GRID_COLS) // 2
TILE_SIZE = 100
BOT_THINK_DELAY_MS = 750
FLIP_BACK_DELAY_MS = 900

# ------------------------- Load user images -------------------------
IMAGE_FILES = [
    "apple.png",
    "orange.png",
    "camera.jpg",
    "basketball.jpg",
    "coconut.png"
]

def load_tile_images(size, pair_count):
    images = []
    all_imgs = []

    for file in IMAGE_FILES:
        try:
            img = Image.open(file).convert('RGBA')
            img = img.resize((size - 10, size - 10), Image.LANCZOS)

            # Create a white background with border
            bg = Image.new('RGBA', (size, size), (255, 255, 255, 255))
            bg.paste(img, ((size - img.width)//2, (size - img.height)//2), img)
            all_imgs.append(ImageTk.PhotoImage(bg))
        except Exception as e:
            print(f"Error loading {file}: {e}")

    if not all_imgs:
        raise RuntimeError("No valid images loaded!")

    for i in range(pair_count):
        images.append(all_imgs[i % len(all_imgs)])

    all_images = images + images[:]  # make pairs
    random.shuffle(all_images)
    return all_images

# ------------------------- Game logic -------------------------
class TileGame:
    def __init__(self, master):
        self.master = master
        self.master.title("Picture Tile Match — Human vs bot")
        self.images = load_tile_images(TILE_SIZE, PAIR_COUNT)

        self.frame = tk.Frame(master, padx=10, pady=10)
        self.frame.pack()

        ctrl = tk.Frame(self.frame)
        ctrl.grid(row=0, column=0, columnspan=GRID_COLS, sticky="we", pady=(0,8))
        self.info_lbl = tk.Label(ctrl, text="Your turn — click to flip a tile", font=(None, 12))
        self.info_lbl.pack(side="left")
        self.reset_btn = tk.Button(ctrl, text="Restart", command=self.reset_game)
        self.reset_btn.pack(side="right")

        self.board_frame = tk.Frame(self.frame)
        self.board_frame.grid(row=1, column=0)

        self.buttons = []
        self.tile_values = []
        self.revealed = []
        self.matched = []
        self.flipped_indices = []

        self.player_score = 0
        self.bot_score = 0
        self.score_lbl = tk.Label(ctrl, text=self._score_text())
        self.score_lbl.pack(side="right", padx=(0,10))

        self.bot_memory = {}
        self.bot_enabled = True
        self.bot_turn = False

        self.setup_board()

    def setup_board(self):
        total = GRID_ROWS * GRID_COLS
        self.tile_values = self.images[:]
        self.revealed = [False] * total
        self.matched = [False] * total

        self.back_image = self._generate_back_tile(TILE_SIZE)

        for idx in range(total):
            row = idx // GRID_COLS
            col = idx % GRID_COLS
            btn = tk.Button(self.board_frame, image=self.back_image, command=partial(self.on_tile_click, idx), bd=2, relief='raised')
            btn.grid(row=row, column=col, padx=6, pady=6)
            self.buttons.append(btn)

    def _generate_back_tile(self, size):
        img = Image.new('RGBA', (size, size), (255,255,255,255))  # completely white
        return ImageTk.PhotoImage(img)

    def on_tile_click(self, idx):
        if self.bot_turn or self.revealed[idx] or self.matched[idx] or len(self.flipped_indices) >= 2:
            return

        self.reveal_tile(idx)

        if len(self.flipped_indices) == 2:
            self.master.after(200, self.check_match, *self.flipped_indices)

    def reveal_tile(self, idx):
        self.revealed[idx] = True
        self.buttons[idx].configure(image=self.tile_values[idx], relief='sunken')
        self.flipped_indices.append(idx)
        img_id = id(self.tile_values[idx])
        self.bot_memory.setdefault(img_id, set()).add(idx)

    def hide_tile(self, idx):
        self.revealed[idx] = False
        self.buttons[idx].configure(image=self.back_image, relief='raised')

    def check_match(self, i1, i2):
        if self.matched[i1] or self.matched[i2]:
            self.flipped_indices.clear()
            return

        same = id(self.tile_values[i1]) == id(self.tile_values[i2])
        if same:
            self.matched[i1] = True
            self.matched[i2] = True
            if self.bot_turn:
                self.bot_score += 1
            else:
                self.player_score += 1
            self.score_lbl.config(text=self._score_text())
            self.info_lbl.config(text=("bot scored!" if self.bot_turn else "Nice! You scored."))
            self.flipped_indices.clear()
            img_id = id(self.tile_values[i1])
            if img_id in self.bot_memory:
                self.bot_memory.pop(img_id, None)
            if sum(self.matched) == len(self.matched):
                self.end_game()
            else:
                if self.bot_turn:
                    self.master.after(BOT_THINK_DELAY_MS, self.bot_move)
        else:
            self.info_lbl.config(text=("bot is thinking..." if self.bot_turn else "Not a match."))
            self.master.after(FLIP_BACK_DELAY_MS, self._flip_back_and_continue)

    def _flip_back_and_continue(self):
        for idx in list(self.flipped_indices):
            if not self.matched[idx]:
                self.hide_tile(idx)
        self.flipped_indices.clear()
        if self.bot_enabled:
            self.bot_turn = not self.bot_turn
            if self.bot_turn:
                self.info_lbl.config(text="bot's turn")
                self.master.after(BOT_THINK_DELAY_MS, self.bot_move)
            else:
                self.info_lbl.config(text="Your turn — click a tile")

    def bot_move(self):
        if not self.bot_turn:
            return

        total = GRID_ROWS * GRID_COLS

        for img_id, indices in list(self.bot_memory.items()):
            unmatched = [i for i in indices if not self.matched[i]]
            if len(unmatched) >= 2:
                i1, i2 = unmatched[0], unmatched[1]
                self.reveal_tile(i1)
                self.master.after(350, lambda: self.reveal_tile(i2))
                self.master.after(550, lambda: self.check_match(i1, i2))
                return

        candidates = [i for i in range(total) if not self.revealed[i] and not self.matched[i]]
        if not candidates:
            return

        first = random.choice(candidates)
        self.reveal_tile(first)

        img_id = id(self.tile_values[first])
        known_positions = [i for i in self.bot_memory.get(img_id, set()) if i != first and not self.matched[i]]
        if known_positions:
            second = known_positions[0]
            self.master.after(350, lambda: self.reveal_tile(second))
            self.master.after(550, lambda: self.check_match(first, second))
            return

        candidates = [i for i in range(total) if i != first and not self.revealed[i] and not self.matched[i]]
        if not candidates:
            self.master.after(200, self._flip_back_and_continue)
            return

        second = random.choice(candidates)
        self.master.after(350, lambda: self.reveal_tile(second))
        self.master.after(650, lambda: self.check_match(first, second))

    def _score_text(self):
        return f"You: {self.player_score}    Bot: {self.bot_score}"

    def end_game(self):
        if self.player_score > self.bot_score:
            winner = "You win! 🎉"
        elif self.player_score < self.bot_score:
            winner = "bot wins — try again!"
        else:
            winner = "It's a tie!"
        messagebox.showinfo("Game over", f"{winner}\nFinal score — {self._score_text()}")
        self.info_lbl.config(text="Game over — hit Restart to play again")

    def reset_game(self):
        for btn in self.buttons:
            btn.destroy()
        self.buttons.clear()
        self.images = load_tile_images(TILE_SIZE, PAIR_COUNT)
        self.bot_memory.clear()
        self.flipped_indices.clear()
        self.player_score = 0
        self.bot_score = 0
        self.score_lbl.config(text=self._score_text())
        self.bot_turn = False
        self.info_lbl.config(text="Your turn — click to flip a tile")
        self.setup_board()

if __name__ == '__main__':
    root = tk.Tk()
    game = TileGame(root)
    root.mainloop()
